In [23]:
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta
import pandas as pd
import yfinance as yf
import requests
import time
import os

Each stock is from 2017 to current because only that much data is available on moneycontrol. Then removing all the news of the same date and consecutive dates because we can't measure the sentiment if more tha 1 news occur at the same time. Then we take difference of opening price of news date and the closing price of next date to find effect of news. Scale it from -5 to 5 for each stock this gives us the sentiment of news.

In [ ]:
Base_url = 'https://www.moneycontrol.com/'
df2 = pd.read_csv('Top_Companies - India.csv')

for x in range(0,df2.shape[0]):
    if (1): 
        z=0
        name = df2['Name'][x]
        site = df2['site'][x]
        tickers = df2['Tickers'][x]
        print(name)
        
        df1 = yf.download(str(tickers)+".NS", start="2017-01-01", end="2021-05-20",group_by="ticker") 
        df1.reset_index(inplace=True)
        
        List_of_links=[]
        all_news = []
        all_headlines = []
        dates1 = []
        
        for i in range(2017,2022):
            for j in range(1,100):
                
                dates2=[]
                sub_links1=[]
                
                html = requests.get('https://www.moneycontrol.com/stocks/company_info/stock_news.php?sc_id='+ str(site)+ '&scat=&pageno='+str(j)+'&next=0&durationType=Y&Year='+str(i)+'&duration=1&news_type=')
                soup = BeautifulSoup(html.text,'html.parser')
                sub_links = soup.find_all('a', class_='arial11_summ')
                if not sub_links:
                    break;
                dates = soup.find_all('p', class_='PT3 a_10dgry')
                dates = str(dates)
                dates = re.findall('[0-9o/]{2}[\s][A-Z]{1}[a-z]{2}[\s][0-9O/]{4}[\s]', dates)
                dates = [x[:-1] for x in dates]
                for m in range(len(dates)):
                    date = dates[m].split(" ")
                    month = {'Jan':'01','Feb':'02','Mar':'03','Apr':'04','May':'05','Jun':'06','Jul':'07','Aug':'08',
                             'Sep':'09','Oct':'10','Nov':'11','Dec':'12' }
                    date[1] = month[date[1]]
                    date.reverse()
                    date = '-'.join(date)
                    date = datetime.strptime(date, '%Y-%m-%d').date()
                    dates2.append(date)
                for k in range(len(dates2)):
                    i=0
                    for l in range(len(dates2)):
                        if dates2[l] == dates2[k] or (dates2[l] - dates2[k]).days == 1:
                            i=i+1
                    if i==1:
                        dates1.append(dates2[k])
                        try:
                            sub_links1.append(sub_links[k])
                        except:
                            z=1
                            break
                if z==1:
                    break
                print(dates1)
                for links in sub_links1:
                    sp = BeautifulSoup(str(links),'html.parser')  
                    tag = sp.a
                    category_links = Base_url + tag["href"]
                    List_of_links.append(category_links)
                    time.sleep(3)

    for links in List_of_links:
        html = requests.get(links)
        soup = BeautifulSoup(html.text,'html.parser')
        Headlines = soup.find_all('title')
        Headlines = str(Headlines)
        Headlines = Headlines.strip("[<title>")
        Headlines = Headlines.strip("/title>]")
        Headlines = Headlines[ : -1]
        all_headlines.append(Headlines)
        if soup.find('div',{'class':'arti-flow'}):
            news_text = soup.find('div',{'class':'arti-flow'})
        else:
            continue
        for x in news_text.find_all("script"):
            x.decompose()
        for y in news_text.find_all('style'):
            y.decompose()
        try:
            news_text.find_all('a')[-1].decompose()
        except:
            all_news.append(Headlines)
            continue
        news = news_text.text
        news = news[news.find('More'):]
        all_news.append(news)
    
    df = pd.DataFrame()
    df = pd.DataFrame(list(zip(dates1, all_headlines, all_news)), columns =['Date', 'Headlines', 'News'])
    df['Date'] = pd.to_datetime(df['Date'])
    days1 = timedelta(1)
    days2 = timedelta(2)
    days3 = timedelta(3)

    Open = []
    Close = []
    for i in range(len(df)):
        if df['Date'][i].weekday() == 5: 
            if df1[df1['Date'] == df['Date'][i]-days1]['Open'].empty == False:
                Open.append(df1[df1['Date'] == df['Date'][i]-days1]['Open'].values[0])
            else:
                Open.append(0)
        elif df['Date'][i].weekday() == 6:        
            if df1[df1['Date'] == df['Date'][i]-days2]['Open'].empty == False:
                Open.append(df1[df1['Date'] == df['Date'][i]-days2]['Open'].values[0])
            else:
                Open.append(0)
        else:
            if df1[df1['Date'] == df['Date'][i]]['Open'].empty == False:
                Open.append(df1[df1['Date'] == df['Date'][i]]['Open'].values[0])
            else:
                Open.append(0)

    for i in range(len(df)):
        if df['Date'][i].weekday() == 4: 
            if df1[df1['Date'] == df['Date'][i] + days3]['Close'].empty == False:
                Close.append(df1[df1['Date'] == df['Date'][i] + days3]['Close'].values[0])
            else:
                Close.append(0)
        elif df['Date'][i].weekday() == 5:
            if df1[df1['Date'] == df['Date'][i] + days2]['Close'].empty == False:
                Close.append(df1[df1['Date'] == df['Date'][i] + days2]['Close'].values[0])
            else:
                Close.append(0)
        else:
            if df1[df1['Date'] == df['Date'][i] + days1]['Close'].empty == False:
                Close.append(df1[df1['Date'] == df['Date'][i] + days1]['Close'].values[0])
            else:
                Close.append(0)
        
    df['Open'] = Open
    df['Close'] = Close

    for i in range(df.shape[0]):    
        if pd.isnull(df['Close'][i]) == True or pd.isnull(df['Open'][i]) == True:
            df = df.drop(i)
            i=i+1    

    df['Diff'] = df['Close'] - df['Open']
    try:
        if min(df['Diff']) < 0 and max(df['Diff']) > 0:
            print('Proceed')
        else:
            print('Check if all positive and negative labels and redefine process')
    except:
        continue
        
    diff1 = abs(min(df['Diff'])/5)
    diff2 = max(df['Diff'])/5
    l1=[]
    for i in range(df.shape[0]):
        if df['Diff'][i] < (min(df['Diff']) + diff1):
            l1.append(-5)
        if df['Diff'][i] >= (min(df['Diff'])+ diff1) and df['Diff'][i] < (min(df['Diff'])+ 2*diff1):
            l1.append(-4)
        if df['Diff'][i] >= (min(df['Diff'])+ 2*diff1) and df['Diff'][i] < (min(df['Diff'])+ 3*diff1):
            l1.append(-3)
        if df['Diff'][i] >= (min(df['Diff'])+ 3*diff1) and df['Diff'][i] < (min(df['Diff'])+ 4*diff1):
            l1.append(-2)
        if df['Diff'][i] >= (min(df['Diff'])+ 4*diff1) and df['Diff'][i] < 0:
            l1.append(-1)
        if df['Diff'][i] == 0:
            l1.append(0)
        if df['Diff'][i] < diff2 and df['Diff'][i] > 0:
            l1.append(1)
        if df['Diff'][i] >= diff2 and df['Diff'][i] < 2*diff2:
            l1.append(2)
        if df['Diff'][i] >= 2*diff2 and df['Diff'][i] < 3*diff2:
            l1.append(3)
        if df['Diff'][i] >= 3*diff2 and df['Diff'][i] < 4*diff2:
            l1.append(4)
        if df['Diff'][i] >= 4*diff2 and df['Diff'][i] <= max(df['Diff']):
            l1.append(5)
    try:
        df['sentiment'] = l1
        df.to_csv(str(name)+'.csv')
    except:
        continue

I have appended 0 where opening and closing price is not available. But this will affect our difference and sentiments of the dataset so I will remove all such values and calculate the sentiment again. Also a little deviation of difference of 0 from both sides considered as 5 sentiment news. 

In [24]:
for filename in os.listdir('Dataset'):
    filename1 = os.path.join('Dataset', filename)
    df = pd.read_csv(str(filename1))
    try:
        df= df[df['Close'] != 0]
    except:
        continue
    df= df[df['Open'] != 0]
    df['Diff'] = df['Close'] - df['Open']
    df.reset_index(drop=True, inplace=True)
    diff1 = abs(min(df['Diff'])/5)
    diff2 = max(df['Diff'])/5
    l1=[]
    for i in range(df.shape[0]):
        if df['Diff'][i] < (min(df['Diff']) + diff1):
            l1.append(-4)
        if df['Diff'][i] >= (min(df['Diff'])+ diff1) and df['Diff'][i] < (min(df['Diff'])+ 2*diff1):
            l1.append(-3)
        if df['Diff'][i] >= (min(df['Diff'])+ 2*diff1) and df['Diff'][i] < (min(df['Diff'])+ 3*diff1):
            l1.append(-2)
        if df['Diff'][i] >= (min(df['Diff'])+ 3*diff1) and df['Diff'][i] < (min(df['Diff'])+ 4*diff1):
            l1.append(-1)
        if df['Diff'][i] >= (min(df['Diff'])+ 4*diff1) and df['Diff'][i] < 0:
            l1.append(0)
        if df['Diff'][i] == 0:
            l1.append(0)
        if df['Diff'][i] < diff2 and df['Diff'][i] > 0:
            l1.append(0)
        if df['Diff'][i] >= diff2 and df['Diff'][i] < 2*diff2:
            l1.append(1)
        if df['Diff'][i] >= 2*diff2 and df['Diff'][i] < 3*diff2:
            l1.append(2)
        if df['Diff'][i] >= 3*diff2 and df['Diff'][i] < 4*diff2:
            l1.append(3)
        if df['Diff'][i] >= 4*diff2 and df['Diff'][i] <= max(df['Diff']):
            l1.append(4)
    
    selected_columns = df[["Date","Headlines","News","sentiment"]]
    df = selected_columns.copy()
    df['sentiment'] = l1
    df.to_csv(str(filename1)) 

In [25]:
global i
i=0
for filename in os.listdir('Dataset'):
    if filename.endswith(".csv"): 
        filename1 = os.path.join('Dataset', filename)
        df = pd.read_csv(str(filename1))
        i = i+df.shape[0]
        
print(i)

8713


In [27]:
df = pd.DataFrame()
for filename in os.listdir('Dataset'):
    filename1 = os.path.join('Dataset', filename)
    df = df.append(pd.read_csv(str(filename1)))
    
df.to_csv('Stocks_Dataset.csv')
df.shape[0]

8713